# Automatic Prompt Tuning

In [69]:
%pip install -q huggingface-hub==0.23.2
%pip install -q transformers==4.47.0
%pip install -q datasets==2.19.1
%pip install -q sentence-transformers==2.7.0
%pip install -q optuna==3.6.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.0 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.23.2 which is incompatible.


## 0. Setup

In [70]:
import os
import yaml
from google.colab import drive
from getpass import getpass

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
# Read YAML file
f_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/private_keys.yml"
with open(f_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
os.environ['HF_API_TOKEN'] = data_loaded['HF_API_KEY']
os.environ['GITHUB_TOKEN'] = data_loaded['GITHUB_TOKEN']

In [72]:
# Set up token
from huggingface_hub import login
login(token=os.environ['HF_API_TOKEN'])

In [73]:
# Example (directly accesing HF endpoint w/ requests)
#model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name = "Qwen/Qwen2.5-72B-Instruct"
API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}

## 1. Initial example

In [74]:
import requests
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from joblib import Parallel, delayed
from sentence_transformers import SentenceTransformer, util

In [85]:
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def get_output(input_prompt, dct_params={'max_new_tokens': 250, 'temperature': 0.1}, mod=True):
  data = query({'inputs': input_prompt, 'parameters': dct_params})
  try:
    data =  data[0]['generated_text']
  except:
    print(data)
    raise ValueError()
  if mod:
    data = data.split(input_prompt)[-1] # Ensure it does not contain the input prompt
  return data

In [86]:
def get_embedding(text):
    """Generate sentence embeddings for a given text."""
    return embedding_model.encode(text, convert_to_tensor=True)

def compute_similarity(output, target):
    """Compute cosine similarity between generated output and target text."""
    output_embedding = get_embedding(output)
    target_embedding = get_embedding(target)
    return util.cos_sim(output_embedding, target_embedding).item()

def optimize_prompt_with_llm(current_prompt, input, output, target_output, similarity, previous_context, dct_params, verbose=False):
    """Optimize the prompt using another LLM."""
    input_text = (
        f"You are an expert prompt engineer. "
        f"You will receive a base prompt used for an LLM (**Current Prompt**), the specific context used for generating the answer (**Input**), the output generated by another LLM (**Generated Output**) and the target output that should have been generated by the LLM (**Target Output**) along with the cosine similarity between the output and the target output (**Cosine Similarity**)."
        f"You will also have other LLM examples within (**Previous context**)."
        f"Your objective is to improve the base **Current Prompt** in order to have a base prompt that yields a **Generated Output** closer to **Target Output**."
        f"This new **Current Prompt** cannot contain specific information from **Input**, **Generated Output**, **Target Output** or **Previous context** since it's the base prompt that would be used for different specific contexts."
        f"Only provide the new **Current Prompt**, nothing else"
        f"* **Current Prompt**: {current_prompt}\n"
        f"* **Input**: {input}\n"
        f"* **Generated Output**: {output}\n"
        f"* **Target Output**: {target_output}\n"
        f"* **Cosine Similarity**: {similarity:.4f}\n"
        f"* **Previous context**: {previous_context}\n"
        f"* **Provide a new **Current Prompt** **:"
    )
    if verbose:
      print(input_text)

    return get_output(input_text, dct_params)

In [77]:
# Embedding model
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(embedding_model_name)

In [78]:
# Load an example dataset for text generation
dataset = load_dataset("xsum", split="test[:100]")  # A dataset with input and target texts

# Base prompt to tune
base_prompt = "Summarize the following article: "

In [79]:
# Prepare dataset articles and targets
articles = dataset["document"][:15]  # Input texts for generation
targets = dataset["summary"][:15]  # Target texts to match

In [80]:
# Hyperparameters for prompt tuning
dct_params={'max_tokens': 250, 'temperature': 0.5, 'top-p': 0.6}
num_iterations = 2

### Without parallelization

In [81]:
optimized_prompt = base_prompt
print(optimized_prompt)
print()

input = articles[0]
target = targets[0]
generated_output = get_output(input, dct_params)
similarity = compute_similarity(generated_output, target)

optimized_prompt = optimize_prompt_with_llm(
  optimized_prompt,
  input=input,
  output=generated_output,
  target_output=target,
  similarity=similarity,
  previous_context="",
  dct_params=dct_params,
  verbose=True
)
#print(optimized_prompt)

Summarize the following article: 

You are an expert prompt engineer. You will receive a base prompt used for an LLM (**Current Prompt**), the specific context used for generating the answer (**Input**), the output generated by another LLM (**Generated Output**) and the target output that should have been generated by the LLM (**Target Output**) along with the cosine similarity between the output and the target output (**Cosine Similarity**).You will also have other LLM examples within (**Previous context**).Your objective is to improve the base **Current Prompt** in order to have a base prompt that yields a **Generated Output** closer to **Target Output**.This new **Current Prompt** cannot contain specific information from **Input**, **Generated Output**, **Target Output** or **Previous context** since it's the base prompt that would be used for different specific contexts.Only provide the new **Current Prompt**, nothing else* **Current Prompt**: Summarize the following article: 
* **

In [82]:
print(optimized_prompt)

 **

"Explain the housing situation for prison leavers in Wales"


In [83]:
input_prompt = f"""
Summarize this text:

Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for accommodation was "chronic".\n"There\'s a desperate need for it, finding suitable accommodation for those leaving prison there is just a lack of it everywhere," he said.\n"It could take six months to a year, without a lot of help they could be on the streets for six months.\n"When you think of the consequences of either being on the street, especially with the cold weather at the moment or you may have a roof over your head, sometimes there is only one choice."\nMr Stevens believes building more one-bedroom flats could help ease the problem.\n"The average price is a hundred pounds a week to keep someone in a rented flat, prison is a lot more than that so I would imagine it would save the public purse quite a few pounds," he said.\nOfficial figures show 830 one-bedroom properties were built in the year to March 2016, of an overall total of 6,900 new properties in Wales.\nMarc, 50, who has been in and out of prison for the past 20 years for burglary offences, said he struggled to find accommodation each time he was released.\nHe said he would ask himself: "Where am I going to stay? Where am I going to live? Have I got somewhere where I can see my daughter."\n"You\'re put out among the same sort of people doing the same sort of thing, and it\'s difficult, it\'s difficult to get away from it. It\'s like every man for himself, there\'s nothing."\nMarc has now found stable accommodation with homeless charity Emmaus and said it had been life changing.\n"You feel safe, you got hot food, you\'ve got company of people in similar situations to yourself but all dealing with different issues. It\'s a constructive, helpful atmosphere," he said.\nTom Clarke, chief executive of Emmaus South Wales, agreed there was not enough support available.\n"We do still see [people] homeless on the streets, so clearly they haven\'t got accommodation and haven\'t got provision," he said.\n"I think the key is connecting people with the services they need. I don\'t delude myself that Emmaus can offer a one size fits all for everyone, we can\'t.\n"But there must be other opportunities and given suitable encouragement I believe that can and should happen."\nA Welsh Government spokesman said the national pathway for homeless services to children, young people and adults in the secure estate had prevented many people from losing their home whilst serving their prison sentence.\nIt added there were already significant demands for one-bedroom flats across the public and private sector and it was providing 20,000 new affordable homes in the next five years

"""
get_output(input_prompt, dct_params=dct_params)

'Prison Link Cymru, a charity that helps ex-offenders find accommodation, had 1,099 referrals in 2015-16. Some ex-offenders were living rough for up to a year before finding suitable accommodation. The charity argues that investment in housing would be cheaper than jailing homeless repeat offenders. The Welsh Government says more people than ever are getting help to address housing problems, but changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation. Prison Link Cymru says things are generally good for women because issues such as children or domestic violence are now considered, but the same cannot be said for men, as issues such as post-traumatic stress disorder or drug dependency are often viewed as less of a priority. Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, says the need for accommodation is "chronic" and building more one-bedroom flats could help ease the p

In [87]:
# Prompt tuning loop
optimized_prompt = base_prompt
previous_context = ""
for iteration in tqdm(range(num_iterations)):
    similarities = []
    for input, target in tqdm(zip(articles, targets), total=len(articles)):
        print(optimized_prompt)
        input_template = f"{optimized_prompt} {input}"

        # Generate output using the current prompt
        generated_output = get_output(input_template, dct_params)

        # Compute similarity between generated output and target
        similarity = compute_similarity(generated_output, target)
        similarities.append(similarity)

        # Store context
        previous_context += f"{optimized_prompt} {input} {target} {generated_output} {similarity}"
        previous_context = previous_context[-1000:] # Last chars

        optimized_prompt = optimize_prompt_with_llm(
            optimized_prompt,
            input=input,
            output=generated_output,
            similarity=similarity,
            target_output=target,
            previous_context=previous_context,
            dct_params=dct_params
            )
print("Final optimized prompt:", optimized_prompt)

  0%|          | 0/15 [00:00<?, ?it/s]

Summarize the following article: 



  7%|▋         | 1/15 [00:00<00:04,  3.05it/s]

 Explain the housing situation for prison leavers in Wales, according to a charity, and mention the potential cost savings of building more one-bedroom flats for this group.**



 13%|█▎        | 2/15 [00:00<00:04,  2.83it/s]

...


 80%|████████  | 12/15 [00:04<00:01,  2.81it/s]



Explain the potential impact of a funding reduction on a university's staffing, student population, and the local economy, considering the severity of the reduction and the university's ability to adapt.


  0%|          | 0/2 [01:04<?, ?it/s]

{'error': 'Model too busy, unable to get response in less than 60 second(s)'}


ValueError: 

In [ ]:
print("Final optimized prompt:", optimized_prompt)

In [ ]:
previous_context

In [ ]:
print(input)
print()
generated_output = get_output(input_template, dct_params, mod=True)
print(generated_output)

print()
out = optimize_prompt_with_llm(
  optimized_prompt,
  input=input,
  output=generated_output,
  similarity=similarity,
  target_output=target,
  previous_context=previous_context,
  dct_params=dct_params,
  verbose=True
  )
print()
print(out)

In [ ]:
"""
# Prompt tuning loop
optimized_prompt = base_prompt
for iteration in tqdm(range(num_iterations)):
    similarities = []

    for input, target in tqdm(zip(articles, targets), total=len(articles)):
        input_template = f"{optimized_prompt} {input}"

        # Generate output using the current prompt
        generated_output = get_output(input_template, dct_params)

        # Compute similarity between generated output and target
        similarity = compute_similarity(generated_output, target)
        similarities.append(similarity)

    # Compute the average similarity for this iteration
    avg_similarity = np.mean(similarities)
    print(f"Iteration {iteration + 1}/{num_iterations}, Avg Similarity: {avg_similarity:.4f}")

    # Update prompt using the optimizer LLM
    if iteration < num_iterations - 1:
        optimized_prompt = optimize_prompt_with_llm(
            optimized_prompt,
            input=articles[0],
            output=get_output(articles[0], dct_params),
            similarity=avg_similarity,
            dct_params=dct_params
            )
print("Final optimized prompt:", optimized_prompt)
"""

In [ ]:
'''
You are an expert prompt engineer.
You will receive a base prompt used for an LLM (**Current Prompt**), the specific context used for generating the answer (**Input**), the output generated by another LLM (**Generated Output**) and the target output that should have been generated by the LLM (**Target Output**) along with the cosine similarity between the output and the target output (**Cosine Similarity**).
You will also have other LLM examples within (**Previous context**).
Your objective is to improve the base **Current Prompt** in order to have a base prompt that yields a **Generated Output** closer to **Target Output**.
This new **Current Prompt** cannot contain specific information from **Input**, **Generated Output**, **Target Output** or **Previous context** since it's the base prompt that would be used for different specific contexts.
Only provide the new *Current Prompt**

* **Current Prompt**: Summarize the following article:
* **Input**: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for accommodation was "chronic".
"There's a desperate need for it, finding suitable accommodation for those leaving prison there is just a lack of it everywhere," he said.
"It could take six months to a year, without a lot of help they could be on the streets for six months.
"When you think of the consequences of either being on the street, especially with the cold weather at the moment or you may have a roof over your head, sometimes there is only one choice."
Mr Stevens believes building more one-bedroom flats could help ease the problem.
"The average price is a hundred pounds a week to keep someone in a rented flat, prison is a lot more than that so I would imagine it would save the public purse quite a few pounds," he said.
Official figures show 830 one-bedroom properties were built in the year to March 2016, of an overall total of 6,900 new properties in Wales.
Marc, 50, who has been in and out of prison for the past 20 years for burglary offences, said he struggled to find accommodation each time he was released.
He said he would ask himself: "Where am I going to stay? Where am I going to live? Have I got somewhere where I can see my daughter."
"You're put out among the same sort of people doing the same sort of thing, and it's difficult, it's difficult to get away from it. It's like every man for himself, there's nothing."
Marc has now found stable accommodation with homeless charity Emmaus and said it had been life changing.
"You feel safe, you got hot food, you've got company of people in similar situations to yourself but all dealing with different issues. It's a constructive, helpful atmosphere," he said.
Tom Clarke, chief executive of Emmaus South Wales, agreed there was not enough support available.
"We do still see [people] homeless on the streets, so clearly they haven't got accommodation and haven't got provision," he said.
"I think the key is connecting people with the services they need. I don't delude myself that Emmaus can offer a one size fits all for everyone, we can't.
"But there must be other opportunities and given suitable encouragement I believe that can and should happen."
A Welsh Government spokesman said the national pathway for homeless services to children, young people and adults in the secure estate had prevented many people from losing their home whilst serving their prison sentence.
It added there were already significant demands for one-bedroom flats across the public and private sector and it was providing 20,000 new affordable homes in the next five years.
* **Generated Output**:
"We recognise that housing can be a key factor in preventing re-offending and that is why we have increased funding for the Supporting People programme by £10m to more than £130m a year," the spokesman said.
"This programme provides support to help people live independently in their own homes, including those leaving prison.
"We are also working with the Ministry of Justice to ensure that the housing needs of those leaving prison are considered as part of their sentence planning."

http://www.bbc.co.uk/news/uk-wales-38054615
* **Target Output**: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
* **Cosine Similarity**: 0.7976
* **Previous context**: ''
* **New **Current Prompt** **:
'''